# Create and run Earthquake Estimator

In [1]:
!nvidia-smi

Fri Mar  8 06:58:37 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:81:00.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import tensorflow as tf
import lstm_estimator
import earthquake_input_fn
import lstm_estimator_4096
from importlib import reload
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path as osp
import pickle
%matplotlib inline

In [3]:
def reload_tf():
    global lstm_estimator
    global earthquake_input_fn
    global lstm_estimator_4096
    for i in range(2):
        import lstm_estimator
        import earthquake_input_fn
        import lstm_estimator_4096
        reload(lstm_estimator)
        reload(earthquake_input_fn)
        reload(lstm_estimator_4096)
        del lstm_estimator
        del earthquake_input_fn
        del lstm_estimator_4096
    import lstm_estimator
    import earthquake_input_fn
    import lstm_estimator_4096
reload_tf()

In [4]:
def create_estimator_4096_from_params(batch_size, timesteps, n_feats, feature_columns,
                                      lstm_cell_size, dense_size, cnn_spec, learning_rate, dropout_rate=0.5, grad_clip=0.5,
                                      time_pool=8, lambda_l2_reg=0.0005, stft_frame_length=512, stft_frame_step=64,
                                      label_input_column=None, model_dir=None, eval_every=600, use_stft=True, use_stride=True,
                                      use_stride_cnn=True, use_stft_cnn=True, regularize_networks=True, use_dense_batch_norm=True,
                                      optimizer_name='Adam', lstm_directionality='bidirectional'):
    
    params = {
        'batch_size': batch_size,
        'timesteps': timesteps,
        'n_feats': n_feats,
        'feature_columns': feature_columns,
        'lstm_cell_size': lstm_cell_size,
        'dense_size': dense_size,
        'cnn_spec': cnn_spec,
        'learning_rate': learning_rate,
        'dropout_rate': dropout_rate,
        'grad_clip': grad_clip,
        'lambda_l2_reg': lambda_l2_reg,
        'stft_frame_length': stft_frame_length,
        'stft_frame_step': stft_frame_step,
        'label_input_column': label_input_column,
        'time_pool': time_pool,
        'use_stft': use_stft,
        'use_stride': use_stride,
        'use_stride_cnn': use_stride_cnn,
        'use_stft_cnn': use_stft_cnn,
        'regularize_networks': regularize_networks,
        'use_dense_batch_norm': use_dense_batch_norm,
        'optimizer_name': optimizer_name,
        'lstm_directionality': lstm_directionality
    }
    
    if model_dir is None:
        model_dir = '/workspace/persistent-data/models/%s' % (time.strftime('%Y-%m-%d-%H-%M-%S'))
    
    params_file = osp.join(osp.split(model_dir)[0], osp.split(model_dir)[1] + '.params.pickle')
    if not osp.isfile(params_file):
        print('writing params to %s' % params_file)
        with open(params_file, 'wb') as f:
            pickle.dump(params, f)
        
    config = tf.estimator.RunConfig(model_dir=model_dir,
                                    log_step_count_steps=int(500 / batch_size),
                                    save_checkpoints_secs=300,
                                    session_config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.75)))
    
    estim = tf.estimator.Estimator(model_fn=lstm_estimator_4096.lstm_4096_model_fn,
                                   params=params,
                                   model_dir=model_dir,
                                   config=config)
    return estim

In [5]:
def train_and_evaluate_estimator_4096(earthquake_data_dir, eval_every_n_secs, eval_count, batch_size,
                                      timesteps, n_feats, feature_columns, lstm_cell_size, dense_size, cnn_spec,
                                      learning_rate, dropout_rate, grad_clip, time_pool, lambda_l2_reg,
                                      stft_frame_length, stft_frame_step, use_stft, use_stride, use_stride_cnn, use_stft_cnn, 
                                      regularize_networks, use_dense_batch_norm, optimizer_name, lstm_directionality,
                                      noise, model_dir=None):
    
    estim = create_estimator_4096_from_params(batch_size, timesteps, n_feats, feature_columns=feature_columns,
                                              lstm_cell_size=lstm_cell_size, dense_size=dense_size, cnn_spec=cnn_spec,
                                              learning_rate=learning_rate, dropout_rate=dropout_rate, grad_clip=grad_clip,
                                              time_pool=time_pool, lambda_l2_reg=lambda_l2_reg, stft_frame_length=stft_frame_length,
                                              stft_frame_step=stft_frame_step, label_input_column=None, model_dir=model_dir,
                                              eval_every=eval_every_n_secs, use_stft=use_stft, use_stride=use_stride,
                                              use_stride_cnn=use_stride_cnn, use_stft_cnn=use_stft_cnn, regularize_networks=regularize_networks,
                                              use_dense_batch_norm=use_dense_batch_norm, optimizer_name=optimizer_name,
                                              lstm_directionality=lstm_directionality)
    
    trainspec = tf.estimator.TrainSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                             batch_size,
                                                                             timesteps,
                                                                             noise,
                                                                             traintest='train',),
                                       max_steps=1000000)
    
    evalspec = tf.estimator.EvalSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                           batch_size,
                                                                           timesteps,
                                                                           noise,
                                                                           traintest='test'),
                                     steps=eval_count,
                                     start_delay_secs=1, throttle_secs=eval_every_n_secs)
    
    return estim, trainspec, evalspec

In [6]:
def build_cnn_spec(cnn_spec_arr):
    return [{'filters':     a[0],
             'kernel_size': a[1],
             'dropout':     a[2],
             'max_pool':    a[3],
             'skip':        a[4],
             'batch_norm':  a[5]}
             for a in cnn_spec_arr]

In [29]:
reload_tf()

EARTHQUAKE_DATA_DIR = '/workspace/persistent-data/earthquake/tfrecords4'
BATCH_SIZE=16
TIMESTEPS=150000
LSTM_CELL_SIZE=[512, 512]
#            Kern Wnd    Drp   Pool  Skip   BatchNorm 
CNN_SPEC = [[64,  15, False, False, False, False],
            [64,  15,  True,  True,  True, False],
            [64,   8, False, False, False, False],
            [64,   8,  True,  True,  True, False],
            [32,   5, False, False, False, False],
            [32,   5, False, False,  True, False],
            [32,   5,  True,  True,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3,  True,  True,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3,  True,  True,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3,  True,  True,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3,  True,  True,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3,  True,  True,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3,  True,  True,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3,  True,  True,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3, False, False,  True, False],
            [32,   3, False,  True,  True, False]]
CNN_SPEC = build_cnn_spec(CNN_SPEC)
DENSE_SIZE=[]
STFT_FRAME_LENGTH=512
STFT_FRAME_STEP=500
LEARNING_RATE=0.001
DROPOUT_RATE=0.5
GRAD_CLIP=1
TIME_POOL=1
LAMBDA_L2_REG=0.0001
EVAL_EVERY_N_SECONDS = 1200
EVAL_NUM_BATCHES = 50000
N_FEATS = 1
USE_STFT = True
USE_STRIDE = False
USE_STRIDE_CNN = True
USE_STFT_CNN = False
REGULARIZE_NETWORKS = False
USE_DENSE_BATCH_NORM = False
NOISE = 1
OPTIMIZER_NAME = 'RMSProp'
LSTM_DIRECTIONALITY = 'unidirectional'
FEATURE_COLUMNS = [tf.feature_column.numeric_column(key='acousticdata', dtype=tf.float32, shape=(TIMESTEPS,))]

estim, train_spec, eval_spec = train_and_evaluate_estimator_4096(EARTHQUAKE_DATA_DIR, EVAL_EVERY_N_SECONDS, EVAL_NUM_BATCHES,
                                                                 BATCH_SIZE, TIMESTEPS, N_FEATS, FEATURE_COLUMNS,
                                                                 LSTM_CELL_SIZE, DENSE_SIZE, CNN_SPEC, LEARNING_RATE,
                                                                 DROPOUT_RATE, GRAD_CLIP, TIME_POOL, LAMBDA_L2_REG,
                                                                 STFT_FRAME_LENGTH, STFT_FRAME_STEP, USE_STFT, USE_STRIDE, USE_STRIDE_CNN,
                                                                 USE_STFT_CNN, REGULARIZE_NETWORKS, USE_DENSE_BATCH_NORM, OPTIMIZER_NAME,
                                                                 LSTM_DIRECTIONALITY, NOISE,
                                                                 #model_dir='/workspace/persistent-data/models/2019-03-08-06-01-14'
                                                                 )

writing params to /workspace/persistent-data/models/2019-03-08-14-10-01.params.pickle
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.75
}
, '_experimental_distribute': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f16f77c65f8>, '_device_fn': None, '_service': None, '_save_summary_steps': 100, '_train_distribute': None, '_protocol': None, '_model_dir': '/workspace/persistent-data/models/2019-03-08-14-10-01', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_eval_distribute': None, '_save_checkpoints_secs': 300, '_master': '', '_is_chief': True, '_num_worker_replicas': 1, '_evaluation_master': '', '_log_step_count_steps': 31, '_task_id': 0, '_global_id_in_cluster': 0}


In [30]:

for i in range(50):
    try:
        print('---------------- BEGIN EPOCH %d ---------------' % i)
        tf.estimator.train_and_evaluate(estim, train_spec, eval_spec)
    except KeyboardInterrupt as e:
        raise e
    #except:
    #    print('end of epoch')

---------------- BEGIN EPOCH 0 ---------------
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Calling model_fn.
FEATURES
{'acousticdata': <tf.Tensor 'IteratorGetNext:0' shape=(?, 150000) dtype=float32>}
--------------------
LABELS
Tensor("IteratorGetNext:1", shape=(?, 150000), dtype=float64, device=/device:CPU:0)
--------------------
MODE
train
--------------------
PARAMS
{'lstm_cell_size': [512, 512], 'learning_rate': 0.001, 'feature_columns': [_NumericColumn(key='acousticdata', shape=(150000,), default_value=None, dtype=tf.float32, normalizer_fn=None)], 'time_pool': 1, 'stft_frame_step': 500, 'cnn_spec': [{'dropout': False, 'max_pool': False, 'kernel_size': 15, 'skip': False, 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



INFO:tensorflow:Summary name VAR_<tf.Variable 'stride_cnn_structures/conv1d_7/bias:0' shape=(32,) dtype=float64_ref> is illegal; using VAR__tf.Variable__stride_cnn_structures/conv1d_7/bias_0__shape__32___dtype_float64_ref_ instead.
INFO:tensorflow:Summary name GRAD_<tf.Variable 'stride_cnn_structures/conv1d_7/bias:0' shape=(32,) dtype=float64_ref> is illegal; using GRAD__tf.Variable__stride_cnn_structures/conv1d_7/bias_0__shape__32___dtype_float64_ref_ instead.
INFO:tensorflow:Summary name VAR_<tf.Variable 'stride_cnn_structures/conv1d_8/kernel:0' shape=(3, 32, 32) dtype=float64_ref> is illegal; using VAR__tf.Variable__stride_cnn_structures/conv1d_8/kernel_0__shape__3__32__32__dtype_float64_ref_ instead.
INFO:tensorflow:Summary name GRAD_<tf.Variable 'stride_cnn_structures/conv1d_8/kernel:0' shape=(3, 32, 32) dtype=float64_ref> is illegal; using GRAD__tf.Variable__stride_cnn_structures/conv1d_8/kernel_0__shape__3__32__32__dtype_float64_ref_ instead.
INFO:tensorflow:Summary name VAR_<tf